In [1]:
!pip install duckdb


[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import duckdb
import time
import pandas as pd

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from py_duckdb.similarity_join import tokenizers
from py_duckdb.similarity_join import jaccard_join, jaccard_join_brute_force
from py_duckdb.similarity_join import evaluate

In [5]:
def join_fn_exec_time(n, join_fn, *args, **kwargs):
    exec_time = []
    for i in range(0, n):
        start_time = time.time()
        join_fn(*args, **kwargs)
        end_time = time.time()
        exec_time.append(end_time - start_time)
        print(end_time - start_time, 's')
    return exec_time

In [6]:
import string
import numpy as np

def test_vs_brute_force(
        con: duckdb.DuckDBPyConnection,
        l_table: string,
        r_table: string,
        l_key_attr: string,
        r_key_attr: string,
        l_join_attr: string,
        r_join_attr: string,
        tokenizer: tokenizers.Tokenizer,
        threshold: float,
        out_table_name: string,
        n=1
):
    exec_times = []
    exec_times_bf = []

    print("FILTERED EXECUTIONS")
    exec_times = join_fn_exec_time(
        n, jaccard_join,
        con, l_table, r_table, l_key_attr, r_key_attr, l_join_attr, r_join_attr, tokenizer, threshold, out_table_name
    )
    print("Average execution time:", np.average(exec_times))

    print()
    print("BRUTE FORCE EXECUTIONS")
    exec_times_bf = join_fn_exec_time(
        n, jaccard_join_brute_force,
        con, l_table, r_table, l_key_attr, r_key_attr, l_join_attr, r_join_attr, tokenizer, threshold, "bf_" + out_table_name
    )
    print("Average execution time:", np.average(exec_times_bf))

    print()
    cmp_join = con.execute(
        "select * "
        f"from {out_table_name} m "
        f"full outer join bf_{out_table_name} b "
        "on (b.rid1 = m.rid1 and b.rid2 = m.rid2) "
        "or (b.rid1 = m.rid2 and b.rid2 = m.rid1) "
        "where m.rid1 is null "
        "or b.rid1 is null"
    ).fetchall()
    if len(cmp_join) == 0:
        print("SUCCESS! Filtered join and Brute force join returned the same result")
    else:
        print("ERROR! There are mismatches between Filtered and Brute force joins:", cmp_join)

    return {
        'exec_time': exec_times,
        'exec_time_bf': exec_times_bf
    }

In [7]:
def test_confusion_mtx(
        con: duckdb.DuckDBPyConnection,
        l_table: string,
        r_table: string,
        l_key_attr: string,
        r_key_attr: string,
        l_join_attr: string,
        r_join_attr: string,
        tokenizer: tokenizers.Tokenizer,
        threshold: float,
        out_table_name: string,
        ground_truth_table: string,
        n=1
):
    exec_time = join_fn_exec_time(
        n, jaccard_join,
        con, l_table, r_table, l_key_attr, r_key_attr, l_join_attr, r_join_attr, tokenizer, threshold, out_table_name
    )
    print("Average execution time:", np.average(exec_time))
    print(evaluate(con, ground_truth_table, out_table_name))
    print()
    return exec_time

In [8]:
con = duckdb.connect(database=':memory:')

# Test case: Actors

In [ ]:
con.execute("drop table if exists src1").execute(
    "CREATE TABLE src1 AS "
    "SELECT id as rid, concat(given_name, ' ', surname, ' ', date_of_birth) as val "
    "FROM 'data/S1_clean_.csv'"
).execute("select * from src1").fetchall()

In [ ]:
con.execute("drop table if exists src2").execute(
    "CREATE TABLE src2 AS "
    "SELECT id as rid, concat(given_name, ' ', surname, ' ', date_of_birth) as val "
    "FROM 'data/S2_clean_.csv'"
).execute("select * from src2").fetchall()

In [ ]:
con.execute("drop table if exists src3").execute(
    "CREATE TABLE src3 AS "
    "SELECT id as rid, concat(given_name, ' ', surname, ' ', date_of_birth) as val "
    "FROM 'data/S3_clean_.csv'"
).execute("select * from src3").fetchall()

In [ ]:
con.execute("drop view if exists srcall").execute(
    "create view srcall as "
    "select * from src1 "
    "union "
    "select * from src2 "
    "union "
    "select * from src3 "
).execute("select * from srcall").fetchall()

In [ ]:
# function args
l_table = 'src1'
r_table = 'src2'

l_key_attr = 'rid'
r_key_attr = 'rid'
l_join_attr = 'val'
r_join_attr = 'val'
# tokenizer = tokenizers.WordsTokzr(r"' '")
tokenizer = tokenizers.QGramsTokzr(3)
threshold = 0.5
out_table_name = 'matches'

In [ ]:
test_vs_brute_force(
    con, 'srcall', '', l_key_attr, r_key_attr, l_join_attr, r_join_attr, tokenizer, threshold, out_table_name, 10
)

In [ ]:
test_vs_brute_force(
    con, 'src1', 'src2', l_key_attr, r_key_attr, l_join_attr, r_join_attr, tokenizer, threshold, out_table_name, 10
)

# Test case: NCVR

In [ ]:
to_concat = ", ' ', ".join(["entity", "rec_id", "first_name", "last_name", "sex", "age", "birth_place", "house_num", "county_desc", "street_name", "zip_code", "phone_num"])
to_concat

In [ ]:
con.execute("drop table if exists src1").execute(
    "CREATE TABLE src1 AS "
    f"SELECT id as rid, concat({to_concat}) as val "
    "FROM 'data/NCVR_AF_clean.csv'"
).execute("select * from src1").fetchall()

In [ ]:
con.execute("drop table if exists src2").execute(
    "CREATE TABLE src2 AS "
    f"SELECT id as rid, concat({to_concat}) as val "
    "FROM 'data/NCVR_BF_clean.csv'"
).execute("select * from src2").fetchall()

In [ ]:
con.execute("drop table if exists src3").execute(
    "CREATE TABLE src3 AS "
    f"SELECT id as rid, concat({to_concat}) as val "
    "FROM 'data/NCVR_CF_clean.csv'"
).execute("select * from src3").fetchall()

In [ ]:
con.execute("drop view if exists srcall").execute(
    "create view srcall as "
    "select * from src1 "
    "union "
    "select * from src2 "
    "union "
    "select * from src3 "
).execute("select * from srcall").fetchall()

In [ ]:
l_key_attr = 'rid'
r_key_attr = 'rid'
l_join_attr = 'val'
r_join_attr = 'val'
tokenizer = tokenizers.WordsTokzr("' '")
# tokenizer = tokenizers.QGramsTokzr(3)
threshold = 0.5
out_table_name = 'matches'

In [ ]:
test_vs_brute_force(
    con, 'srcall', '', l_key_attr, r_key_attr, l_join_attr, r_join_attr, tokenizer, threshold, out_table_name, 10
)

In [ ]:
test_vs_brute_force(
    con, 'src1', 'src2', l_key_attr, r_key_attr, l_join_attr, r_join_attr, tokenizer, threshold, out_table_name, 10
)

In [ ]:
con.execute("drop table if exists src1")
con.execute("drop table if exists src2")
con.execute("drop table if exists src3")
con.execute("drop view if exists srcall")

## Test Case: Profiles

In [9]:
df10 = pd.read_json("data/10Kprofiles.json", lines=True, orient='records', typ='frame')

In [10]:
df10.head()

,realProfileID,date_of_birth,surname,address_1,street_number,postcode,soc_sec_id,suburb,phone_number,state,given_name,age,address_2
0,0,19390609.0,bishop,daley crescent,41.0,6050,4676841,batlow,08 29028996,qld,molly,31.0,NaN
1,1,19041109.0,aidon,nambucca street,7.0,2002,3414163,devonort,08 75629459,vkf,whkt,NaN,NaN
2,2,19910711.0,anns,NaN,12.0,2287,7844876,ivanhoe,02 11684110,vic,andrew,31.0,NaN
3,3,19390709.0,whitrlsy,robson street,34.0,4065,2418360,christie downs,04 00323207,qld,shsne,NaN,NaN
4,4,19340328.0,roche,rankin street,9.0,3644,7577436,frenchs forest,08 53227250,qld,sophie,30.0,NaN


In [11]:
to_concat = ", ' ', ".join(["date_of_birth", "surname", "address_1", "street_number", "postcode", "soc_sec_id", "suburb", "phone_number", "state", "given_name", "age", "address_2"])
to_concat

"date_of_birth, ' ', surname, ' ', address_1, ' ', street_number, ' ', postcode, ' ', soc_sec_id, ' ', suburb, ' ', phone_number, ' ', state, ' ', given_name, ' ', age, ' ', address_2"

In [12]:
con.execute("drop table if exists db10").execute(
    "CREATE TABLE db10 AS "
    f"SELECT realProfileID as rid, concat ({to_concat}) as val "
    "FROM df10"
).execute("select * from db10").fetchall()

[(0,
  '19390609.0 bishop daley crescent 41.0 6050 4676841 batlow 08 29028996 qld molly 31.0 '),
 (1,
  '19041109.0 aidon nambucca street 7.0 2002 3414163 devonort 08 75629459 vkf whkt  '),
 (2,
  '19910711.0 anns  12.0 2287 7844876 ivanhoe 02 11684110 vic andrew 31.0 '),
 (3,
  '19390709.0 whitrlsy robson street 34.0 4065 2418360 christie downs 04 00323207 qld shsne  '),
 (4,
  '19340328.0 roche rankin street 9.0 3644 7577436 frenchs forest 08 53227250 qld sophie 30.0 '),
 (5,
  '19320811.0 fullgrabe beeston street 29.0 3131 6494586 broken hill 04 80080021 nsw emma 29.0 '),
 (6,
  '19601013.0 lodge mason street 48.0 5254 6098877 orchard hills 08 48143359 vic rourke 32.0 '),
 (7,
  '19340921.0 coleman edman close 116.0 2397 8858237 castlecrag 03 58778382  harry 26.0 '),
 (8,
  '19671108.0 bishop cromwell circuit 12.0 2226 1718686 harris park 04 01707833 vic jamie 28.0 homestead caravan park'),
 (9,
  '19331228.0 grcn lutana street 11.0 2074 8364236 newstead 02 57700508 ws lauren  '),
 

In [13]:
df10gt = pd.read_json("data/10KIdDuplicates.json", lines=True, orient='records', typ='frame')
df10gt.head()

,d1Id,d2Id
0,101,8933
1,4101,4614
2,7213,8466
3,4856,7485
4,4829,9725


In [14]:
con.execute("drop table if exists db10gt").execute(
    "CREATE TABLE db10gt AS "
    "SELECT d1Id as rid1, d2Id as rid2 "
    "FROM df10gt "
).execute("select count(*) from db10gt").fetchall()

[(8705,)]

In [15]:
l_key_attr = 'rid'
r_key_attr = 'rid'
l_join_attr = 'val'
r_join_attr = 'val'
out_table_name = 'matches'

In [16]:
con.execute("drop table if exists db10_sample").execute(
    "CREATE TABLE db10_sample AS "
    "SELECT * "
    "FROM db10 "
    "using sample 2000"
).execute("select * from db10_sample").fetchall()

[(6066,
  '19060920.0 vett ogilby crescent 15.0 4121 2794760 lavington 08 41750984 vic kristen 24.0 antelle park'),
 (2143,
  '19171915.0 cleavr-smith somerville street 5.0 2647 3967764 ballatat 07 76952064 nsw edward  terra nostra'),
 (7197,
  '19181202.0 dent box hill avenue  2232 4757811 knoxfield 03 73379375  allea 31.0 wirrabilla'),
 (8950,
  '19410321.0 mercorella jervois street 71.0 2650 1374650 terrigal 03 34995345 sa sana 30.0 brentwood vlge'),
 (7301,
  ' matthews dentliace 5.0 3779 8582946 west lakes shore 08 79995631 nsw atj 40.0 '),
 (4042,
  '19090115.0 jervis smalley circuit 7.0 7004 7766121 yorkeys knob 08 45280716 nsw isabella 28.0 hangmans lane'),
 (6331,
  '19851127.0 losgw le gallienne street 50.0 3037 8448576 cranboure west  gic chloe 30.0 east end'),
 (7315,
  '19300626.0 matson bannister garden 498.0 3162 8357227 surrey hills  sa maya 25.0 glenferrie park'),
 (4076,
  '19270620.0 hanna moulden court 259.0 3044 4324624 mosman 07 37151298 nsw trent 19.0 shopping vi

In [17]:
con.execute("drop view if exists db10gt_sample").execute(
    "create view db10gt_sample as "
    "select gt.* "
    "from db10_sample s1, db10_sample s2, db10gt gt "
    "where s1.rid = gt.rid1 "
    "and s2.rid = gt.rid2"
).execute("select count(*) from db10gt_sample").fetchall()

[(358,)]

In [21]:
test_vs_brute_force(
    con, 'db10_sample', '', l_key_attr, r_key_attr, l_join_attr, r_join_attr,
    tokenizers.WordsTokzr("' '"),
    0.2,
    out_table_name, 5
)
evaluate(con, 'db10gt_sample', out_table_name)

FILTERED EXECUTIONS
0.2728464603424072 s
0.24100089073181152 s
0.27881693840026855 s
0.22381162643432617 s
0.26530933380126953 s
Average execution time: 0.2563570499420166

BRUTE FORCE EXECUTIONS
0.42899179458618164 s
0.6743893623352051 s
0.54398512840271 s
0.41567063331604004 s
0.4300065040588379 s
Average execution time: 0.4986086845397949

SUCCESS! Filtered join and Brute force join returned the same result


{'tp': 350,
 'fp': 50,
 'fn': 58,
 'pr': 0.875,
 'rc': 0.8578431372549019,
 'fm': 0.8663366336633663}

In [23]:
test_confusion_mtx(
    con, 'db10_sample', '', l_key_attr, r_key_attr, l_join_attr, r_join_attr,
    tokenizers.QGramsTokzr(5),
    0.2,
    out_table_name, 'db10gt_sample', 5
)

1.6381425857543945 s
1.3475825786590576 s
1.3402888774871826 s
1.2615272998809814 s
1.2226409912109375 s
Average execution time: 1.3620364665985107
{'tp': 329, 'fp': 1, 'fn': 30, 'pr': 0.996969696969697, 'rc': 0.9164345403899722, 'fm': 0.9550072568940494}



[1.6381425857543945,
 1.3475825786590576,
 1.3402888774871826,
 1.2615272998809814,
 1.2226409912109375]

In [ ]:
for t in [0.6, 0.5, 0.4, 0.3, 0.2]:
    print("threshold =", t)
    test_confusion_mtx(con, 'db10_sample', '', l_key_attr, r_key_attr, l_join_attr, r_join_attr,
          # tokenizers.QGramsTokzr(5),
          tokenizers.WordsTokzr("' '"),
          t, out_table_name, "db10gt_sample", 5)

In [25]:
for t in [0.6, 0.5, 0.4, 0.3, 0.2]:
    print("threshold =", t)
    test_confusion_mtx(con, 'db10', '', l_key_attr, r_key_attr, l_join_attr, r_join_attr,
          tokenizers.QGramsTokzr(5),
          # tokenizers.WordsTokzr("' '"),
          t, out_table_name, "db10gt", 5)

threshold = 0.6
2.518188953399658 s
2.435110330581665 s
2.455605983734131 s
1.9553611278533936 s
1.4302787780761719 s
Average execution time: 2.158909034729004
{'tp': 487, 'fp': 0, 'fn': 8218, 'pr': 1.0, 'rc': 0.055944859276278, 'fm': 0.10596170583115753}

threshold = 0.5
1.9692881107330322 s
1.3780174255371094 s
1.858508586883545 s
3.167698383331299 s
3.166992425918579 s
Average execution time: 2.3081009864807127
{'tp': 1928, 'fp': 0, 'fn': 6777, 'pr': 1.0, 'rc': 0.22148190695002873, 'fm': 0.3626445970093107}

threshold = 0.4
22.384418725967407 s
22.137258529663086 s
20.2163143157959 s
19.777273178100586 s
22.051724195480347 s
Average execution time: 21.313397789001463
{'tp': 3942, 'fp': 0, 'fn': 4763, 'pr': 1.0, 'rc': 0.45284319356691555, 'fm': 0.6233889459950976}

threshold = 0.3
17.752160787582397 s
14.252793073654175 s
21.12443780899048 s
22.04925799369812 s
21.43597388267517 s
Average execution time: 19.32292470932007
{'tp': 6343, 'fp': 0, 'fn': 2362, 'pr': 1.0, 'rc': 0.728661688

## Test case: larger Profiles datasets

In [ ]:
df50 = pd.read_json("data/50Kprofiles.json", lines=True, orient='records', typ='frame')

In [ ]:
con.execute("drop table if exists src2").execute(
    "CREATE TABLE src2 AS "
    f"SELECT realProfileID as rid, concat ({to_concat}) as val "
    "FROM df50"
).execute("select * from src2").fetchall()

In [ ]:
df100 = pd.read_json("data/100Kprofiles.json", lines=True, orient='records', typ='frame')

In [ ]:
con.execute("drop table if exists src3").execute(
    "CREATE TABLE src3 AS "
    f"SELECT realProfileID as rid, concat ({to_concat}) as val "
    "FROM df100"
).execute("select * from src3").fetchall()

In [ ]:
start_time = time.time()
jaccard_join(
    con, 'src2', '', l_key_attr, r_key_attr, l_join_attr, r_join_attr, tokenizers.WordsTokzr("' '"), 0.5, out_table_name
)
time.time() - start_time

In [ ]:
start_time = time.time()
jaccard_join(
    con, 'src3', '', l_key_attr, r_key_attr, l_join_attr, r_join_attr, tokenizers.QGramsTokzr("' '"), 0.5, out_table_name
)
time.time() - start_time